# Fermi-Hubbard Model

In [ ]:
using Pkg; Pkg.activate()
using KadanoffBaym
using LinearAlgebra, BlockArrays
using UnPack
using JLD

In [ ]:
using PyPlot
PyPlot.plt.style.use("./paper.mplstyle")

## Model

### Hamiltonian

$$
\begin{align}\begin{split}
    \hat{H} &= - J \sum_{\langle{i,\,j}\rangle}\sum_\sigma \hat{c}^{\dagger}_{i,\sigma} \hat{c}^{\phantom{\dagger}}_{i+1,\sigma} + U\sum_{i=1}^L  \hat{c}^{\dagger}_{i,\uparrow} \hat{c}^{\phantom{\dagger}}_{i,\uparrow}   \hat{c}^{\dagger}_{i,\downarrow} \hat{c}^{\phantom{\dagger}}_{i,\downarrow}, 
\end{split}\end{align}
$$

### Green functions

$$
    G^>_{ij}(t, t') = -i \left\langle \hat{c}^{\phantom{\dagger}}_{i,\uparrow}(t) \hat{c}^{{\dagger}}_{i,\uparrow}(t') \right\rangle\\
    F^>_{ij}(t, t') = -i \left\langle \hat{c}^{\phantom{\dagger}}_{i,\downarrow}(t) \hat{c}^{{\dagger}}_{i,\downarrow}(t') \right\rangle\\
$$

### Self-energies

$$
    \Sigma^{\mathrm{HF}}_{\uparrow,\,ij}(t, t') = {\mathrm{i}}\delta_{ij}\delta(t - t') F^<_{ii}(t, t)\\
    \Sigma^{\mathrm{HF}}_{\downarrow,\,ij}(t, t') = {\mathrm{i}}\delta_{ij}\delta(t - t') G^<_{ii}(t, t)
$$

$$
    \Sigma^{\mathrm{NCA}}_{\uparrow,\,ij}(t, t') = U^2 G_{ij}(t, t') F_{ij}(t, t') F_{ji}(t', t)\\
    \Sigma^{\mathrm{NCA}}_{\downarrow,\,ij}(t, t') = U^2 F_{ij}(t, t') G_{ij}(t, t') G_{ji}(t', t)
$$

## Solving

In [ ]:
num_sites = 8;

In [ ]:
# 8-site 3D cubic lattice
h = BlockArray{ComplexF64}(undef_blocks, [4, 4], [4, 4])
diag_block = [0 -1 0 -1; -1 0 -1 0; 0 -1 0 -1; -1 0 -1 0]
setblock!(h, diag_block, 1, 1)
setblock!(h, diag_block, 2, 2)
setblock!(h, Diagonal(-1 .* ones(4)), 1, 2)
setblock!(h, Diagonal(-1 .* ones(4)), 2, 1)

full_h = BlockArray{ComplexF64}(undef_blocks, [8, 8], [8, 8])
setblock!(full_h, h |> Array, 1, 1)
setblock!(full_h, h |> Array, 2, 2)
setblock!(full_h, zeros(ComplexF64, 8, 8), 1, 2)
setblock!(full_h, zeros(ComplexF64, 8, 8), 2, 1)

full_h = full_h |> Array;

In [ ]:
Base.@kwdef struct FermiHubbardModel
    U::Float64
    
    # 8-site 3D cubic lattice
    H = begin
        h = BlockArray{ComplexF64}(undef_blocks, [4, 4], [4, 4])
        diag_block = [0 -1 0 -1; -1 0 -1 0; 0 -1 0 -1; -1 0 -1 0]
        setblock!(h, diag_block, 1, 1)
        setblock!(h, diag_block, 2, 2)
        setblock!(h, Diagonal(-1 .* ones(4)), 1, 2)
        setblock!(h, Diagonal(-1 .* ones(4)), 2, 1)

        full_h = BlockArray{ComplexF64}(undef_blocks, [8, 8], [8, 8])
        setblock!(full_h, h |> Array, 1, 1)
        setblock!(full_h, h |> Array, 2, 2)
        setblock!(full_h, zeros(ComplexF64, 8, 8), 1, 2)
        setblock!(full_h, zeros(ComplexF64, 8, 8), 2, 1)
        
        full_h |> Array
    end
    
    H1 = H[1:8, 1:8]
    H2 = H[1 + 8:2 * 8, 1 + 8:2 * 8]
end

In [ ]:
struct FermiHubbardData{T}
    GL::T
    GG::T
    FL::T
    FG::T

    ΣNCA_up_L::T
    ΣNCA_up_G::T
    ΣNCA_down_L::T
    ΣNCA_down_G::T

    # Initialize problem
    function FermiHubbardData(GL::T, GG::T, FL::T, FG::T) where {T}
        new{T}(GL, GG, FL, FG, zero(GL), zero(GG), zero(FL), zero(FG))
    end
end

In [ ]:
begin
    function integrate1(hs::Vector, t1, t2, A::GreenFunction, B::GreenFunction, C::GreenFunction; tmax=t1)
        retval = zero(A[t1,t1])

        @inbounds for k in 1:tmax
            @views LinearAlgebra.mul!(retval, A[t1, k] - B[t1, k], C[k, t2], hs[k], 1.0)
        end
        return retval
    end

    function integrate2(hs::Vector, t1, t2, A::GreenFunction, B::GreenFunction, C::GreenFunction; tmax=t2)
        retval = zero(A[t1,t1])

        @inbounds for k in 1:tmax
            @views LinearAlgebra.mul!(retval, A[t1, k], B[k, t2] - C[k, t2], hs[k], 1.0)
        end
        return retval
    end
end;

In [ ]:
function fv!(model, data, out, times, h1, h2, t, t′)
    @unpack GL, GG, FL, FG, ΣNCA_up_L, ΣNCA_up_G, ΣNCA_down_L, ΣNCA_down_G = data
    @unpack H1, H2, U = model

    # real-time collision integral
    ∫dt1(A,B,C) = integrate1(h1, t, t′, A, B, C)#sum(h1[s] * ((A[t, s] - B[t, s]) * C[s, t′]) for s in 1:t)
    ∫dt2(A,B,C) = integrate2(h2, t, t′, A, B, C)#sum(h2[s] * (A[t, s] * (B[s, t′] - C[s, t′])) for s in 1:t′)
    
    ΣHF_c(t, t′) = 1.0im * U * Diagonal(FL[t, t])
    ΣHF_f(t, t′) = 1.0im * U * Diagonal(GL[t, t])
    
    out[1] = -1.0im * ((H1 + ΣHF_c(t, t′)) * GL[t, t′] + 
            ∫dt1(ΣNCA_up_G, ΣNCA_up_L, GL) + ∫dt2(ΣNCA_up_L, GL, GG)
        )

    out[2] = -1.0im * ((H1 + ΣHF_c(t, t′)) * GG[t, t′] + 
            ∫dt1(ΣNCA_up_G, ΣNCA_up_L, GG) + ∫dt2(ΣNCA_up_G, GL, GG)
        )

    out[3] = -1.0im * ((H2 + ΣHF_f(t, t′)) * FL[t, t′] + 
            ∫dt1(ΣNCA_down_G, ΣNCA_down_L, FL) + ∫dt2(ΣNCA_down_L, FL, FG)
        )

    out[4] = -1.0im * ((H2 + ΣHF_f(t, t′)) * FG[t, t′] +
            ∫dt1(ΣNCA_down_G, ΣNCA_down_L, FG) + ∫dt2(ΣNCA_down_G, FL, FG)
        )    
    return out
end

function fd!(model, data, out, times, h1, h2, t, t′)
    fv!(model, data, out, times, h1, h2, t, t)
    out .-= adjoint.(out)
end

function self_energies!(model, data, times, h1, h2, t, t′)
    @unpack GL, GG, FL, FG, ΣNCA_up_L, ΣNCA_up_G, ΣNCA_down_L, ΣNCA_down_G = data
    @unpack U = model

    if (n = size(GL, 3)) > size(ΣNCA_up_L, 3)
        resize!(ΣNCA_up_L, n)
        resize!(ΣNCA_up_G, n)
        resize!(ΣNCA_down_L, n)
        resize!(ΣNCA_down_G, n)
    end

    ΣNCA_up_L[t, t′] = U^2 .* GL[t, t′] .* FL[t, t′] .* transpose(FG[t′, t])
    ΣNCA_up_G[t, t′] = U^2 .* GG[t, t′] .* FG[t, t′] .* transpose(FL[t′, t])

    ΣNCA_down_L[t, t′] = U^2 .* FL[t, t′] .* GL[t, t′] .* transpose(GG[t′, t])
    ΣNCA_down_G[t, t′] = U^2 .* FG[t, t′] .* GG[t, t′] .* transpose(GL[t′, t])
end

In [ ]:
# quantum numbers
dim = 8

# Allocate the initial Green functions (time arguments at the end)
GL = GreenFunction(zeros(ComplexF64, dim, dim, 1, 1), SkewHermitian)
GG = GreenFunction(zeros(ComplexF64, dim, dim, 1, 1), SkewHermitian)
FL = GreenFunction(zeros(ComplexF64, dim, dim, 1, 1), SkewHermitian)
FG = GreenFunction(zeros(ComplexF64, dim, dim, 1, 1), SkewHermitian)

# Initial condition
N_up = zeros(8)
N_down = zeros(8)
N_up[1:4] = [0.7, 0.0, 0.7, 0.0]
N_down[1:4] = [0.0, 0.25, 0.0, 0.25]

N_up[5:8] = [0.0, 0.4, 0.0, 0.4]
N_down[5:8] = [0.65, 0.0, 0.65, 0.0]

GL[1, 1] = 1.0im * diagm(N_up)
GG[1, 1] = -1.0im * (I - diagm(N_up))
FL[1, 1] = 1.0im * diagm(N_down)
FG[1, 1] = -1.0im * (I - diagm(N_down))

data = FermiHubbardData(GL, GG, FL, FG)
model = FermiHubbardModel(U = 0.25)

tmax = 32;

In [ ]:
atol = 1e-8
rtol = 1e-6;

In [ ]:
@time sol = kbsolve!(
    (x...) -> fv!(model, data, x...),
    (x...) -> fd!(model, data, x...),
    [data.GL, data.GG, data.FL, data.FG],
    (0.0, tmax);
    callback = (x...) -> self_energies!(model, data, x...),
    atol = atol,
    rtol = rtol,
    stop = x -> (println("t: $(x[end])"); flush(stdout); false)
);

In [ ]:
save("FH_3D_sol_U_"*string(model.U)*"_tmax_"*string(tmax)*"_atol_"*string(atol)*"_rtol_"*string(rtol)*".jld", "solution", sol)

## Example plots

### Load data

In [ ]:
loaded_sol = load("FH_3D_sol_U_"*string(model.U)*"_tmax_"*string(tmax)*"_atol_"*string(atol)*"_rtol_"*string(rtol)*".jld")
num_points = (loaded_sol["solution"].t |> size)[1];

In [ ]:
import PyCall
const is = PyCall.pyimport("mpl_toolkits.axes_grid1.inset_locator")
const inset_axes = is.inset_axes

In [ ]:
xpad = 8
ypad = 5

fig, (ax1, ax2) = subplots(1, 2, figsize = (8, 3))
idx_1 = 1
idx_2 = 8

ax1.plot(loaded_sol["solution"].t, [imag(loaded_sol["solution"].u[1][idx_1, idx_1, k, k] .+ loaded_sol["solution"].u[3][idx_1, idx_1, k, k]) for k = 1:num_points], 
    label = "\$ i=1\$", lw=1.5, ls = "--", c = "#438E6A")

ax1.plot(loaded_sol["solution"].t, [imag(loaded_sol["solution"].u[1][idx_2, idx_2, k, k] .+ loaded_sol["solution"].u[3][idx_2, idx_2, k, k]) for k = 1:num_points], 
    label = "\$ i=8\$", lw=1.5, ls = "-", c = "#2D5FAA")

ax1.set_xlim(0, tmax)
ax1.set_xticks([0, 8, 16, 24, 32])
ax1.set_ylim(-0, 1)
ax1.set_xlabel("\$J t\$")
ax1.set_ylabel("Charge", labelpad = 8)
ax1.xaxis.set_tick_params(pad = xpad)
ax1.yaxis.set_tick_params(pad = ypad)
ax1.legend(loc = "best", handlelength = 1.9, frameon = false, borderpad = 0, labelspacing = 0.25)

# ax2 = subplot(122)

ax2.plot(loaded_sol["solution"].t, [imag(loaded_sol["solution"].u[1][idx_1, idx_1, k, k] .- loaded_sol["solution"].u[3][idx_1, idx_1, k, k]) for k = 1:num_points], 
    label = "\$ i=2\$", lw=1.5, ls = "--", c = "#438E6A")

ax2.plot(loaded_sol["solution"].t, [imag(loaded_sol["solution"].u[1][idx_2, idx_2, k, k] .- loaded_sol["solution"].u[3][idx_2, idx_2, k, k]) for k = 1:num_points], 
    label = "\$ i=2\$",  lw=1.5, ls = "-", c = "#2D5FAA")

ax2.set_xlim(0, tmax)
ax2.set_ylim(-1, 1)
ax2.set_xticks([0, 8, 16, 24, 32])
ax2.set_xlabel("\$J t\$")
ax2.set_ylabel("Spin", labelpad = 16)
ax2.xaxis.set_tick_params(pad = xpad)
ax2.yaxis.set_tick_params(pad = ypad)
ax2.set_axisbelow(false)
ax2.yaxis.set_label_position("right")

axins1 = inset_axes(ax1, width=1.1 * 1.0, height=0.8*3/4, loc=4, 
    bbox_to_anchor=(0.96, .0, .0, .0),
    bbox_transform=ax1.transAxes)

axins1.plot(loaded_sol["solution"].t, [(sum(Gs.u[1][i, i, k, k] for i = 1:num_sites) |> imag) .+ (sum(Gs.u[3][i, i, k, k] for i = 1:num_sites) |> imag) for k = 1:num_points] 
    .- sum(N_up .+ N_down)  .|> abs, 
    label = "\$ c \$", ls = "-", c = "k")
axins1.set_xlim(0, tmax)
axins1.set_xticks([0, 8, 16, 24, 32])
axins1.set_xticklabels([])
axins1.set_yticks([k for k in 0:0.5:1] .* 5e-15)
axins1.set_ylim([0.0, 1] .* 5e-15)
axins1.set_ylabel(L"Q(t) - Q_0", fontdict = Dict(:fontsize=>10))
axins1.tick_params(axis="y", labelsize=10)
axins1.yaxis.get_offset_text().set_fontsize(10)
axins1.yaxis.set_label_position("right")

axins2 = inset_axes(ax2, width=1.1 * 1.0, height=0.8*3/4, loc=4, 
    bbox_to_anchor=(0.96, .0, .0, .0),
    bbox_transform=ax2.transAxes)

axins2.plot(loaded_sol["solution"].t, [(sum(Gs.u[1][i, i, k, k] for i = 1:num_sites) |> imag) .- (sum(Gs.u[3][i, i, k, k] for i = 1:num_sites) |> imag) for k = 1:num_points] 
    .- sum(N_up .- N_down) .|> abs, 
    label = "\$ c \$", ls = "-", c = "k")
axins2.set_xlim(0, tmax)
axins2.set_xticks([0, 8, 16, 24, 32])
axins2.set_yticks([k for k in 0:0.5:1] .* 3e-15)
axins2.set_xticklabels([])
axins2.set_ylim([0.0, 1] .* 3e-15)
axins2.set_ylabel(L"S(t) - S_0", fontdict = Dict(:fontsize=>10))
axins2.tick_params(axis="y", labelsize=10)
axins2.yaxis.get_offset_text().set_fontsize(10)
axins2.yaxis.set_label_position("right")

tight_layout(pad = 0.1, w_pad = 0.5, h_pad = 0)
savefig("fermi_hubbard_T.pdf")

In [ ]:
using FFTW, Interpolations

In [ ]:
# quantum number to look at
idx = 5
shift = 0

ρτ, (τs, ts) = wigner_transform_itp((loaded_sol["solution"].u[2][idx, idx, :, :] - loaded_sol["solution"].u[1][idx, idx, :, :]), 
    loaded_sol["solution"].t[1+shift:end-shift], fourier=false);
ρω, (ωs, ts) = wigner_transform_itp((loaded_sol["solution"].u[2][idx, idx, :, :] - loaded_sol["solution"].u[1][idx, idx, :, :]), 
    loaded_sol["solution"].t[1+shift:end-shift], fourier=true);

In [ ]:
t_scale = 1
ω_scale = 1;

In [ ]:
function meshgrid(xin,yin)
  nx=length(xin)
  ny=length(yin)
  xout=zeros(ny,nx)
  yout=zeros(ny,nx)
  for jx=1:nx
      for ix=1:ny
          xout[ix,jx]=xin[jx]
          yout[ix,jx]=yin[ix]
      end
  end
  return (x=xout, y=yout)
end

Y, X = meshgrid(loaded_sol["solution"].t, loaded_sol["solution"].t);

In [ ]:
cmap = "gist_heat";

In [ ]:
figure(figsize=(7, 3))
t_scale = 1
vmin = -1.0
vmax = 1.0

center = floor(length(ts) / 2) |> Int

ax = subplot(121)
plot(t_scale * τs, -ρτ[:, center] |> imag, ls="-", c="C0", lw=1.5)
ax.set_xlabel("\$J \\tau\$")
ax.set_xlim(-t_scale * tmax / 2, t_scale * tmax / 2)
ax.set_ylim(-1, 1)
ax.set_xticks(t_scale .* [-tmax/2, -tmax/4, 0, tmax/4, tmax/2])
ax.xaxis.set_tick_params(pad=xpad)
ax.yaxis.set_tick_params(pad=ypad)
ax.set_ylabel("\$  A_{11, \\uparrow}(T_{\\mathrm{max}}/2, \\tau)_W \$")

ax = subplot(122)
heatmap = ax.pcolormesh(X, Y, imag(loaded_sol["solution"].u[1][1, 1, :, :]) .- imag(loaded_sol["solution"].u[2][1, 1, :, :]), cmap=cmap, rasterized=true, vmin=vmin, vmax=vmax)
heatmap.set_edgecolor("face")
ax.set_aspect("equal")
cbar = colorbar(mappable=heatmap)
cbar.formatter.set_powerlimits((0, 0))
ax.set_xlabel("\$J t\$")
ax.set_ylabel("\$J t'\$")
ax.set_xlim(0, t_scale * tmax)
ax.set_ylim(0, t_scale * tmax)
ax.set_xticks(t_scale .* [0, tmax/2, tmax])
ax.set_yticks(t_scale .* [0, tmax/2, tmax])

tight_layout(pad=0.75, w_pad=0.5, h_pad=0)

# savefig("fermi_hubbard_example_two_times.pdf")